In [1]:
import torch
import torch.nn as nn  # neural network modules
import torch.nn.functional as F  # activation functions
import torch.optim as optim  # optimizer
from torch.autograd import Variable # add gradients to tensors
from torch.nn import Parameter # model parameter functionality 

import pandas as pd
import numpy as np
import csv
import itertools

In [2]:
# Read in data
df = pd.read_csv("train.csv")

# Fix target label
label_encodings = {
    'pants-fire': 0, 
    'false':      0, 
    'barely-true':1, 
    'half-true':  1, 
    'mostly-true':2,
    'true':       2
}
df['target'] = df['label'].apply(lambda x: label_encodings[x])

In [3]:
pt = torch.load('train_bert.pt')

### FakeBERT

In [49]:
def get_accuracy(output, targets):

    predicted = [int(y_pred.detach().argmax(-1)) for y_pred in output]
    targets = [int(y) for y in targets]
    correct = sum(a==b for (a,b) in zip(predicted, targets))
    accuracy = 100*correct/len(targets) 

    return accuracy

def train(data_X,
          test_X,
          data_y,
          test_y,
          num_epochs = 10,
          batch_size = 100,
          learning_rate = 0.01):
    
    # Instantiate model & optimization 
    model = FakeBERT()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
    # Prepare test data
    test_y = torch.Tensor(test_y.values).to(dtype=torch.long)
    
    # Iterate over epochs
    for ep in range(num_epochs):
        
        model.train()

        # Iterate over batches
        for i in range(data_X.shape[0]//batch_size):
            # Reset gradients
            optimizer.zero_grad()
            
            # Declare features and target labels
            X = data_X[i*batch_size:(i+1)*batch_size]
            y = data_y[i*batch_size:(i+1)*batch_size].values
            y = torch.Tensor(y).to(dtype=torch.long)

            # Get predictions from model
            pred = model(X)

            # Calculate loss
            loss_func = nn.CrossEntropyLoss()
            loss = loss_func(pred, y)

            # Backpropagate
            loss.backward()

            # Update parameters
            optimizer.step()

        # Evaluate model
        model.eval()
            
        # Evaluate on test data
        test_pred = model(test_X)
        test_accuracy = get_accuracy(test_pred, test_y)
        
        # Print accuracy
        print(f"Test accuracy: {test_accuracy} at epoch: {ep}")

    return test_pred

In [50]:
class FakeBERT(nn.Module):
    def __init__(self):
        super(FakeBERT, self).__init__()
        
        # Layer 1: Conv1D + Maxpool
        self.conv_1 = nn.Conv1d(in_channels=25, out_channels=32, kernel_size=3, stride=1)
        self.sigm_1 = nn.ReLU()
        self.pool_1 = nn.MaxPool1d(kernel_size=5, stride=5)
        
        # Layer 2: Conv1D + Maxpool
        self.conv_2 = nn.Conv1d(in_channels=25, out_channels=32, kernel_size=4, stride=1)
        self.sigm_2 = nn.ReLU()
        self.pool_2 = nn.MaxPool1d(kernel_size=5, stride=5)
        
        # Layer 3: Conv1D + Maxpool
        self.conv_3 = nn.Conv1d(in_channels=25, out_channels=32, kernel_size=5, stride=1)
        self.sigm_3 = nn.ReLU()
        self.pool_3 = nn.MaxPool1d(kernel_size=5, stride=5)
        
        # Layer 4: Conv1D + Maxpool
        self.conv_4 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=5, stride=1)
        self.sigm_4 = nn.ReLU()
        self.pool_4 = nn.MaxPool1d(kernel_size=5, stride=5)
        
        # Layer 5: Conv1D + Maxpool
        self.conv_5 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=5, stride=1)
        self.sigm_5 = nn.ReLU()
        self.pool_5 = nn.MaxPool1d(kernel_size=5, stride=5)
        
        # Layer 6: Fully Connected Layer 
        self.full_6 = nn.Linear(544,32)
        self.sigm_6 = nn.Sigmoid()
        
        # Layer 7: Fully Connected Layer 
        self.full_7 = nn.Linear(32,3)
        self.soft_7 = nn.Softmax()
        
    def forward(self, x):
        # Turn into tensor
        x = torch.Tensor(x)
        
        # Generate the 3 1D conv layers
        conv_1 = self.pool_1(self.sigm_1(self.conv_1(x)))        
        conv_2 = self.pool_2(self.sigm_2(self.conv_2(x)))        
        conv_3 = self.pool_3(self.sigm_3(self.conv_3(x)))
        
        # Concatenate the 3 layers
        cat = torch.cat((conv_1,conv_2,conv_3),2)
        
        # Pass the concatenated output through 2 1D conv layers
        conv_4 = self.pool_4(self.sigm_4(self.conv_4(cat)))        
        conv_5 = self.pool_5(self.sigm_5(self.conv_5(conv_4)))  

        # Flatten the output
        flat = conv_5.flatten(start_dim=1)

        # Pass through 2 fully connected layers
        full_6 = self.sigm_6(self.full_6(flat))
        full_7 = self.soft_7(self.full_7(full_6))
        
        return full_7

In [51]:
# data_X = torch.transpose(pt[:10000],1,2)
# test_X = torch.transpose(pt[10000:],1,2)

data_X = pt[:10000]
test_X = pt[10000:]
data_y = df['target'][:10000]
test_y = df['target'][10000:]

In [52]:
test_data = train(data_X, test_X, data_y, test_y)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:59: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Test accuracy: 35.0 at epoch: 0
Test accuracy: 35.0 at epoch: 1
Test accuracy: 35.0 at epoch: 2
Test accuracy: 35.0 at epoch: 3
Test accuracy: 35.0 at epoch: 4
Test accuracy: 35.0 at epoch: 5
Test accuracy: 35.0 at epoch: 6
Test accuracy: 35.0 at epoch: 7
Test accuracy: 35.0 at epoch: 8
Test accuracy: 35.0 at epoch: 9
